In [30]:
import bs4 as bs
import pickle
import requests
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web

# Pretend use browser
headers = {'User-Agent': 'Mozilla/75.0 Chrome/80.0.3987.163 '}

In [32]:
#  stock symbol
def save_dj30_symbols():
    
    # crawler websites
    resp = requests.get('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average',headers = headers)
    
    # source is text, parser is lxml
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    
    # table
    table = soup.find_all("table",{"id" : "constituents"})[0]
    
    
    symbols = []
    
    #tr is row of table
    for row in table.findAll('tr')[1:]:
        
        # soup object get text)
        symbol = row.findAll('td')[0].get_text(strip=True)

        symbols.append(symbol)
        
    #  save result to pickle
    with open("dj30.pickle","wb") as file:
        pickle.dump(symbols, file)
    
    return symbols


In [33]:
save_dj30_symbols()

['3M',
 'American Express',
 'Apple Inc.',
 'Boeing',
 'Caterpillar Inc.',
 'Chevron Corporation',
 'Cisco Systems',
 'The Coca-Cola Company',
 'Dow Inc.',
 'ExxonMobil',
 'Goldman Sachs',
 'The Home Depot',
 'IBM',
 'Intel',
 'Johnson & Johnson',
 'JPMorgan Chase',
 "McDonald's",
 'Merck & Co.',
 'Microsoft',
 'Nike',
 'Pfizer',
 'Procter & Gamble',
 'Raytheon Technologies',
 'The Travelers Companies',
 'UnitedHealth Group',
 'Verizon',
 'Visa Inc.',
 'Walmart',
 'Walgreens Boots Alliance',
 'The Walt Disney Company']

In [39]:
# save price from spider
def get_data_from_datareader(beg_y, beg_m, beg_d, end_y, end_m, end_d, reload_dj30=False):

    # if reload, execute save_dj30_symbols()
    if reload_dj30:
        symbols = save_dj30_symbols()
    # if no reload, read dj30.pickle     
    else:
        with open("dj30.pickle","rb") as file:
            symbols = pickle.load(file)
    
    # if no folder exist, mkdir
    if not os.path.exists('dj30'):
        os.makedirs('dj30')
        
    beg = dt.datetime(beg_y, beg_m, beg_d)
    end = dt.datetime(end_y, end_m, end_d)
#     print(beg,end)

    #for loop price from 'stooq'
    for symbol in symbols:
        #try: check symbol is downloaed
        try:
            if not os.path.exists('dj30/{}.csv'.format(symbol)):
                df = web.DataReader(symbol, 'stooq', beg, end)
                df.to_csv('dj30/{}.csv'.format(symbol))
            else:
                print('already have {}'.format(symbol))
        # except
        except:
            print(symbol + "can not download")
    print("crawler done")

In [41]:
get_data_from_datareader(2020,1,1,2020,4,17)

crawler done
